#### Twitter Mining Project
Course: CISD 43 <br>
Professor: Sohair Zaki <br>
Student: Jack Chen <br>
<br>
Purpose:<br>
- Exercise twitter api, clean text data, visualize data, implement natural language processing (NLP)<br>

Tasks:<br>
- Pre-Setup
- Twitter API
- Cleaning Phase
- Visualization

<br>
Conclusion:

#### Pre-Setup: Loading Data, Functions, etc

In [141]:
# Importing Libraries
# import webbrowser
# import time
import tweepy
from tweepy import Stream
from tweepy.auth import OAuthHandler
import os
import pandas as pd
import re
import glob

# load keys into keys
#[apiKey,apiKeySecret,bearerTokem,accessToken,accessTokenSecret]
# Consumer Keys: apiKey, apiKeySecret
# Access Keys: accessToken, accessTokenSecret
path =  r'C:\Users\Gumo\Desktop\Git\selfBooks\tweepy.txt'
keys = []
with open(path, mode='r') as w:
    for line in w:
        keys.append(line.split(': ')[1].strip())
access_token = keys[3]
access_secret = keys[4]
consumer_key = keys[0]
consumer_secret = keys[1]
bearer_token = keys[2]

###### my id/name #####
myUserId = 1309643764172947456
myUsername = 'stockjanitor'


In [142]:
###### FUNCTIONS ######
def tweetPost(text):
    try:
        twitterApi.update_status(text)
        print("bling bling~~")
    except:
        print("boom boom")
def tweetMedia(text, img):
    try:
        media = twitterApi.media_upload(img)
        twitterApi.update_status(text,media_ids=[media.media_id_string])
        print("bling bling~~")
    except:
        print("boom boom")
def tweetFriend(name):
    try:
        twitterApi.create_friendship(screen_name=name)
        print("bling bling~~")
    except:
        print("boom boom")
def tweetUnfriend(name):
    try:
        twitterApi.destroy_friendship(screen_name=name)
        print("bling bling~~")
    except:
        print("boom boom")

# Regex Remove functions
# remove url
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
#remove hashtag #
def remove_hashtag(txt):
    return " ".join(re.sub("([#＃]+)([0-9A-Z_]*[A-Z_]+[a-z0-9_üÀ-ÖØ-öø-ÿ]*)", "", txt).split())
# remove mention @
def remove_at(txt):
    return " ".join(re.sub("(\@[a-zA-Z0-9_%]*)", "", txt).split())

In [7]:
# Twitter API v1 - auth keys (Cursor)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_secret)

# create instance
twitterApi = tweepy.API(auth, wait_on_rate_limit=True)
# wait on rate limit, not to get errors on timing out

# verify credential
try:
    # see if credential works
    twitterApi.verify_credentials()
    print("twitterApi works")
except:
    print("Please fix me")

# Twitter API v2 - auth keys (Client)
# for user access, tweeting etc
client = tweepy.Client(consumer_key=consumer_key,consumer_secret=consumer_secret,access_token=access_token,access_token_secret=access_secret)

# for query access, search tweets etc
clientBearer = tweepy.Client(bearer_token=bearer_token)


twitterApi works


#### Twitter API to pull data

##### Client - v2

In [8]:
# # store responses of query1 in response1
# response1 = clientBearer.search_all_tweets(query=query1, max_results=10, start_time= "2022-03-28T12:12:01Z", end_time='2022-03-27T13:12:01Z',tweet_fields=['id','text', 'created_at','geo','source', 'public_metrics'])


# # convert response into list1
# list1 = [[tweet.id,tweet.text,tweet.created_at,tweet.geo,tweet.source,tweet.public_metrics]for tweet in response1.data]

# # load to df
# # df = original df 
# df = pd.DataFrame(data=list1,columns=['userid','text','time','location','source','metrics'])

# # show df
# df.head(3)

In [10]:
# # assign new df for editing
# df1 = df

# # extract public metrics into respective columns
# df1['retweet_count'] = df.metrics.apply(lambda x: list(x.items())[0][1])
# df1['reply_count'] = df.metrics.apply(lambda x: list(x.items())[1][1])
# df1['like_count']= df.metrics.apply(lambda x: list(x.items())[2][1])
# df1['quote_count']= df.metrics.apply(lambda x: list(x.items())[3][1])

# # drop the metrics since we have extracted it
# df1 = df1.drop(columns='metrics')

# #show df
# df1.head(3)

##### Cursor v1

In [125]:
# set query1
query1 = "chrisrock"
hour = '07'

date = '20220422'
time =hour+'00'
time2=hour+'59'
fromdate =date+time
todate=date+time2

######################## Search 30 Day Queries ########################
# # store search in tweets
# queryResponse1 = twitterApi.search_30_day(label='30Days',query=query1,fromDate=fromdate,toDate=todate,maxResults=100)
# # save tweets to list1
# list1 = [[tweet.user.screen_name,tweet.text,tweet.created_at,tweet.user.location,tweet.source,tweet.favorite_count,tweet.quote_count,tweet.reply_count,tweet.retweet_count]for tweet in queryResponse1]
# # convert tweet list into dataframe
# df = pd.DataFrame(data=list1,columns=['user','text','time','location','source','like_count','quote_count','reply_count','retweet_count'])
# # save to dataframe
# df.to_csv('data/'+date+time+time2+query1+'.csv')
# df.shape



######################## Search Recent queries ########################
# sincedate=''
# queryResponse1 = twitterApi.search_tweets(q=query1,until=sincedate,count=100)
# # items = 5, retrieve 5 tweets
# list1 = [[tweet.user.screen_name,tweet.text,tweet.created_at,tweet.user.location,tweet.source,tweet.favorite_count,tweet.retweet_count]for tweet in queryResponse1]
# # convert tweet list into dataframe
# df = pd.DataFrame(data=list1,columns=['user','text','time','location','source','like_count','retweet_count'])
# # save to dataframe
# df.to_csv('data/'+sincedate+query1+'.csv')
# df.shape


(97, 7)

#### Cleaning Phase: Load Data, Clean Data

In [143]:
pathOfData = 'data/' 
# saves path all file name to variable all_files
all_files = glob.glob(os.path.join(pathOfData , "*.csv"))
#initiate a list
merged_list1 = []

# loop all files to append each file to list
for a in all_files:
    df = pd.read_csv(a, index_col=None, header=0)
    merged_list1.append(df)

# concat all files
merged_df1 = pd.concat(merged_list1, axis=0, ignore_index=True)
merged_df1.shape

(4970, 10)

In [145]:
pathOfData = 'data2/' 
# saves path all file name to variable all_files
all_files2 = glob.glob(os.path.join(pathOfData , "*.csv"))
#initiate a list
merged_list2 = []

# loop all files to append each file to list
for b in all_files2:
    df = pd.read_csv(b, index_col=None, header=0)
    merged_list2.append(df)

# concat all files
merged_df2 = pd.concat(merged_list2, axis=0, ignore_index=True)
merged_df2.shape

(297, 8)

In [146]:
merged_df2.head(3)

,Unnamed: 0,user,text,time,location,source,like_count,retweet_count
0,0,larepublica_pe,RT @LRTendencias: 🎬 😕 Primero #Netflix y ahora...,2022-04-22 18:24:18+00:00,"Lima, Perú",Twitter Web App,0,1
1,1,AdetoroMph,#ChrisRock: Where was #WillSmith when #MeekMil...,2022-04-22 18:22:40+00:00,"Princeton, NJ",Twitter for iPhone,0,0
2,2,Melisa_Porte,@chrisrock Who's bored her too?,2022-04-22 18:21:49+00:00,NaN,Twitter Web App,0,0


In [147]:
merged_df1.head(3)

,Unnamed: 0,user,text,time,location,source,like_count,quote_count,reply_count,retweet_count
0,0,Hjr0223,It’s the King and Queen #WillSmith and #Jadas...,2022-03-27 23:50:18+00:00,#SOSCuba🇨🇺,Twitter Web App,1,0,0,0
1,1,RevistaHogar,"#JadaSmith, la esposa de Will Smith, eligió el...",2022-03-27 23:49:24+00:00,"Guayaquil, Ecuador",Twitter Web App,0,0,0,0
2,2,Imeraldgurl,#Oscarsredcarpet #Oscars #JadaSmith is there a...,2022-03-27 23:48:41+00:00,"Los Angeles, Ca",Twitter for iPhone,0,0,0,0


In [152]:
# Clean df
df3 = df2.text.apply(remove_hashtag)
df3 = df3.apply(remove_at)
df3 = df3.apply(remove_url)
print(df3)

0    Scorchio south of the Forth today Bring it on ...
1    it could help to create the conditions for hea...
2    AQI 69 Advisory Moderate Air quality is accept...
3    Difficile de liker ce type de post rares sont ...
4    RT Distance between and our sun is about 150 m...
5    RT Lets make this world a better place to live...
6         RT9 ForPlanet Action Crisis Emergency Strike
7    RT Methane emissions 14 minutes must watch Cri...
8    Did you know Crisis Emergency ActionNow shorta...
9                 RT wildfire Crisis Warming ActionNow
Name: text, dtype: object
